**Google Advance Data Analytics Professional Certificate** - Final project

**Waze workplace scenario**

Main goal: Prevent user churn on the Waze app. Churn quantifies the number of users who have uninstalled the Waze app or stopped using the app.

*Developing a churn prediction model will help prevent churn, improve user retention, and grow Waze’s business.*

Who are the users most likely to churn?

Why do users chur? 

When do users chrn? 

**Step 1**

To get clear insights, the data must be inspected, organized, and prepared for EDA.

In [363]:
#IMPORT PACKAGES
import pandas as pd
import numpy as np

#LOAD DATASET INTO DATAFRAME
data = pd.read_csv('waze_dataset.csv')

#LAST MONTH USER INFOMATION
#SUMMARY
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       14999 non-null  int64  
 1   label                    14299 non-null  object 
 2   sessions                 14999 non-null  int64  
 3   drives                   14999 non-null  int64  
 4   total_sessions           14999 non-null  float64
 5   n_days_after_onboarding  14999 non-null  int64  
 6   total_navigations_fav1   14999 non-null  int64  
 7   total_navigations_fav2   14999 non-null  int64  
 8   driven_km_drives         14999 non-null  float64
 9   duration_minutes_drives  14999 non-null  float64
 10  activity_days            14999 non-null  int64  
 11  driving_days             14999 non-null  int64  
 12  device                   14999 non-null  object 
dtypes: float64(3), int64(8), object(2)
memory usage: 1.5+ MB


In [381]:
#DATASET COLUMN EXPLANATION
data_explanation = pd.read_csv('dataset_explanation.csv')
pd.set_option('max_colwidth',None)
data_explanation

,column name,type,description
0,label,obj,Binary target variable (“retained” vs “churned”) for if a user has churned anytime during the course of the month
1,sessions,int,The number of occurrence of a user opening the app during the month
2,drives,int,An occurrence of driving at least 1 km during the month
3,device,obj,The type of device a user starts a session with
4,total_sessions,float,A model estimate of the total number of sessions since a user has onboarded
5,n_days_after_onboarding,int,The number of days since a user signed up for the app
6,total_navigations_fav1,int,Total navigations since onboarding to the user’s favorite place 1
7,total_navigations_fav2,int,Total navigations since onboarding to the user’s favorite place 2
8,driven_km_drives,float,Total kilometers driven during the month
9,duration_minutes_drives,float,Total duration driven in minutes during the month


In [292]:
#700 NULLS VALUES IN LABEL COLUMN
#DIFFERENCE BETWEEN THE TWO POPULATIONS

#SUMMARY INFORMATION OF ROWS WITH NULL VALUES
null_data=data[data['label'].isnull()]
null_data.describe().iloc[1:3].round(2)

,ID,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days
mean,7405.58,80.84,67.80,198.48,1709.30,118.72,30.37,3935.97,1795.12,15.38,12.13
std,4306.90,79.99,65.27,140.56,1005.31,156.31,46.31,2443.11,1419.24,8.77,7.63


In [382]:
#SUMMARY INFORMATION OF ROWS WITHOUT NULL VALUES

notnull_data=data[data['label'].notnull()]
notnull_data.describe().iloc[1:3].round(2)

,ID,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days
mean,7503.57,80.62,67.26,189.55,1751.82,121.75,29.64,4044.40,1864.20,15.54,12.18
std,4331.21,80.74,65.95,136.19,1008.66,147.71,45.35,2504.98,1448.01,9.02,7.83


In [345]:
#THE MEANS AND STANDARD DEVIATIONS ARE CONSISTENT
#NO REMARKABLE DIFFERENCE BETWEEN THE TWO GROUPS

#COUNT OF NULL VALUES BY DEVICE
null_data.groupby(['device']).size()

device
Android    253
iPhone     447
dtype: int64

In [262]:
null_data['device'].value_counts(normalize=True).mul(100).round(2).astype(str)+'%'

device
iPhone     63.86%
Android    36.14%
Name: proportion, dtype: object

In [271]:
#63.86% OF NULL VALUES BELONG TO ADROID USERS
#HOWEVER, THIS VALUE IS CONSISTENT WITH THE OVERALL DATA
data['device'].value_counts(normalize=True).mul(100).round(2).astype(str)+'%'

device
iPhone     64.48%
Android    35.52%
Name: proportion, dtype: object

In [343]:
#CALCULATE SIZE OF CHURNED VS. RETAINED
data.groupby(['label']).size()

label
churned      2536
retained    11763
dtype: int64

In [283]:
#DATA SET CONTAINS 82% RETAINED USERS AND 18% CHURNED USERS
data['label'].value_counts(normalize=True).mul(100).round(2).astype(str)+'%'

label
retained    82.26%
churned     17.74%
Name: proportion, dtype: object

In [314]:
#CALCULATE MEDIAN OF BOTH GROUPS TO EXCLUDE OUTLIERS
data.groupby(['label']).median(numeric_only=True).round(2)

,ID,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days
label,,,,,,,,,,,
churned,7477.5,59.0,50.0,164.34,1321.0,84.5,11.0,3652.66,1607.18,8.0,6.0
retained,7509.0,56.0,47.0,157.59,1843.0,68.0,9.0,3464.68,1458.05,17.0,14.0


In [325]:
#CHURNED USERS HAD MORE DRIVES IN FEWER DAYS, AND THEIR TRIPS WERE FARTHER AND LONGER IN DURATION
#CALCULATE MEDIAN KILOMETERS PER DRIVE

data_label = data.groupby(['label']).median(numeric_only=True)
kilometers_per_drive = data_label['driven_km_drives']/data_label['drives']
kilometers_per_drive.round(2).astype(str)+'%'

label
churned     73.05%
retained    73.72%
dtype: object

In [331]:
#MEDIAN USER FROM BOTH GROUPS DROVE ~73km/drive
#CALCULATE KILOMETERS PER DAYS

kilometers_per_day = data_label['driven_km_drives']/data_label['driving_days']
kilometers_per_day

label
churned     608.775944
retained    247.477472
dtype: float64

In [ ]:
#MEDIAN USER WHO CHURNED DROVE 608km PER DAY
#ALMOST 250% DISTANCE OF RETAINED USERS
#THE NEEDS OF CHURNED USERS DIFFER FROM THE NEEDS OF A MORE TYPICAL DRIVER

In [358]:
#FOR EACH LABEL, CALCULATE THE NUMBER OF ANDROID USERS VS. IPHONE USERS
data.groupby(['label','device']).size()

label     device 
churned   Android     891
          iPhone     1645
retained  Android    4183
          iPhone     7580
dtype: int64

In [361]:
#THE RATIO OF ANDROID AND IPHONE IS CONSISTENT BETWEEN BOTH GROUPS AND THE OVERALL DATASET
data.groupby('label')['device'].value_counts(normalize=True).mul(100).round(2).astype(str)+'%'

label     device 
churned   iPhone     64.87%
          Android    35.13%
retained  iPhone     64.44%
          Android    35.56%
Name: proportion, dtype: object

**SUMMARY**

The dataset has 700 missing values in the label column. There was no obvious pattern to the missing values.

Median was chosen to prevent the influence of outliers.

The median user who churned drove 608 kilometers each day, which is almost 250% the per-drive-day distance of retained users.

Android users comprised approximately 36% of the sample, while iPhone users made up about 64%. 

However, there is nothing suggestive of churn being correlated with device.